# Tischendorf MorphGNT to Text-Fabric

In [1]:
import os
import re
import collections
import json
import csv
from glob import glob
from tf.fabric import Fabric
from tf.convert.walker import CV
from tf.compose import modify

## Path Configs

In [2]:
source_repo = os.path.expanduser('~/github/tischendorf-data')
source_dirs = os.path.join(source_repo, 'word-per-line/*')
output_dirs = '../tf/{version}'

## Get Latest Source Data

Pull the latest Tischendorf data from the [source Github directory](https://github.com/morphgnt/tischendorf-data).

In [3]:
!cd $source_repo; git pull origin master

From https://github.com/morphgnt/tischendorf-data
 * branch            master     -> FETCH_HEAD
Already up to date.


## Select Versions

At this stage, I will seek to convert the latest version of Tischendorf.

In [4]:
versions = sorted(glob(source_dirs))
version_dirs = [versions[-1]] # convert latest version only for now

## Processing Data

Intelligence needed to process the text.

In [5]:
bo2book = {line.split()[0]:line.split()[1] for line in '''
MT Matthew
MR Mark
LU Luke
JOH John
AC Acts
RO Romans
1CO 1_Corinthians
2CO 2_Corinthians
GA Galatians
EPH Ephesians
PHP Philippians
COL Colossians
1TH 1_Thessalonians
2TH 2_Thessalonians
1TI 1_Timothy
2TI 2_Timothy
TIT Titus
PHM Philemon
HEB Hebrews
JAS James
1PE 1_Peter
2PE 2_Peter
1JO 1_John
2JO 2_John
3JO 3_John
JUDE Jude
RE Revelation
'''.split('\n') if line}

patts = {'section': re.compile('(\d*):(\d*)\.(\d*)')}

## Write a CV Walk Director to Process the Text

See the documentation for the CV walk class [here](https://annotation.github.io/text-fabric/Create/Convert/).

In [6]:
def director(cv):
        
    '''
    Walks through Tischendorf and triggers
    slot and node creation events.
    '''
        
    # process books in order
    for bo, book in bo2book.items():
        
        book_loc = os.path.join(version_loc, f'Unicode/{bo}.txt')
        
        print(f'\thandling {book_loc}...')
        
        with open(book_loc, 'r') as infile:
            text = [w for w in infile.read().split('\n') if w]
            
        this_book = cv.node('book')
        cv.feature(this_book, book=book)
            
        # keep track of when to trigger paragraph, chapter, and verse objects
        para_track = 1 # keep counts of paragraphs
        prev_chap = 1 # start at 1
        prev_verse = 1 # start at 1
        this_chap = cv.node('chapter')
        this_para = cv.node('paragraph')
        this_verse = cv.node('verse')
        
        # iterate through words and construct objects
        for word in text:
            
            data = word.split()
            word_data, lemmas = data[:7], data[7:]
            
            # segment out word data
            bo_code, ref, brake, ketiv, qere, morph, strongs = word_data
            strongs_lemma, anlex_lemma = ' '.join(lemmas).split('!') # reconstitute lemmas and split on !

            chapt, verse, wrdnum = [int(v) for v in patts['section'].match(ref).groups()]
            
            # -- handle TF events --
            
            # detect chapter boundary
            if prev_chap != chapt:
                
                # end verse
                cv.feature(this_verse, verse=prev_verse)
                cv.terminate(this_verse)
                
                # end chapter
                cv.feature(this_chap, chapter=prev_chap)
                cv.terminate(this_chap)
                
                # new chapter and verse begin
                this_chap = cv.node('chapter')
                prev_chap = chapt
                this_verse = cv.node('verse')
                prev_verse = verse
            
            # detect verse boundary
            elif prev_verse != verse:
                cv.feature(this_verse, verse=prev_verse)
                cv.terminate(this_verse)
                this_verse = cv.node('verse') # start a new verse
                prev_verse = verse
                
            # detect paragraph boundary
            if brake == 'P':
                cv.feature(this_para, para=para_track)
                cv.terminate(this_para)
                this_para = cv.node('paragraph') # start a new paragraph
                para_track += 1 # count paragraphs in the book
                
            # make word object
            this_word = cv.slot()
            cv.feature(this_word, 
                       ketiv=ketiv, 
                       qere=qere, 
                       morph=morph, 
                       strongs=strongs, 
                       vrsnum=wrdnum,
                       str_lem=strongs_lemma.strip(),
                       anlex_lem=anlex_lemma.strip()
                      )
            cv.terminate(this_word)
        
        # end book and its objects
        # - end verse
        cv.feature(this_verse, verse=prev_verse)
        cv.terminate(this_verse)
        
        # - end paragraph
        cv.feature(this_para, para=para_track)
        cv.terminate(this_para)
        
        # - end chapter
        cv.feature(this_chap, chapter=prev_chap)
        cv.terminate(this_chap)
        
        # - end book
        cv.feature(this_book, book=book, book_code=bo)
        cv.terminate(this_book)

## Make the Conversion

### Corpus and TF Feature Metadata

In [7]:
slotType = 'word'
otext = {'fmt:text-orig-full':'{qere} ',
         'sectionTypes':'book,chapter,verse',
         'sectionFeatures':'book,chapter,verse'}

generic = {'Name': 'Morph-GNT Tischendorf',
           'Version': None, # to be filled in
           'Author': 'Constantin von Tischendorf',
           'Editors': 'Ulrik Sandborg-Petersen, G. Clint Yale, and Maurice A. Robinson',
           'Converter': 'Cody Kingham', 
           'Source:':'https://github.com/morphgnt/tischendorf-data/',
           'Note':'Feature descriptions adapted from tischendorf-data README'}

intFeatures = {'chapter', 'para', 'verse'}

featureMeta = {'book': {'description': 'A book name'},
               'chapter': {'description': 'A chapter number'},
               'verse': {'description': 'A verse number'},
               'book_code':{'description': 'Short book abbreviation'},
               'para': {'description': 'A paragraph number'},
               'ketiv': {'descrption': 'The text as it is written in the printed Tischendorf'},
               'qere': {'description': 'The text as the editor thinks it should have been'},
               'morph': {'description': 'Word morphological tag based on Maurice A Robinson\'s analysis'},
               'strongs': {'description': 'A word\'s number in Strongs'},
               'vrsnum': {'description': 'N-word in verse'},
               'str_lem': {'description': 'Word lemma that corresponds to The NEW Strong\'sComplete Dictionary of Bible Words'},
               'anlex_lem': {'description': 'Word lemma that corresponds to Friberg, Friberg and Miller\'s ANLEX'}
              }

### Conversion

In [8]:
for version_loc in version_dirs:
    
    # configure metadata/output
    version = os.path.basename(version_loc)
    generic['Version'] = version
    
    output = os.path.join(output_dirs, version)

    print(f'Processing Version {version}')
    output_dir = output_dirs.format(version=version)

    TF = Fabric(locations=output_dir, silent=True)
    cv = CV(TF)
    
    good = cv.walk(director,
                   slotType,
                   otext=otext,
                   generic=generic,
                   intFeatures=intFeatures,
                   featureMeta=featureMeta,
                   warn=True,
                   force=False,)

Processing Version 2.8
  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |     0.00s No structure nodes will be set up
   |   SECTION   TYPES:    book, chapter, verse
   |   SECTION   FEATURES: book, chapter, verse
   |   STRUCTURE TYPES:    
   |   STRUCTURE FEATURES: 
   |   TEXT      FEATURES:
   |      |   text-orig-full       qere
   |     0.01s OK
   |     0.00s Following director... 
	handling /Users/cody/github/tischendorf-data/word-per-line/2.8/Unicode/MT.txt...
	handling /Users/cody/github/tischendorf-data/word-per-line/2.8/Unicode/MR.txt...
	handling /Users/cody/github/tischendorf-data/word-per-line/2.8/Unicode/LU.txt...
	handling /Users/cody/github/tischendorf-data/word-per-line/2.8/Unicode/JOH.txt...
	handling /Users/cody/github/tischendorf-data/word-per-line/2.8/Unicode/AC.txt...
	handling /Users/cody/github/tischendorf-data/word-per-line/2.8/Unicode/RO.txt...
	handling /Users/cody/github/tischendorf-data/word-per-line/2.8

## Enhancements

The following new node types and features will be added to the corpus:

* gloss - glosses are added from Mounce
* lex - a node that contains words with the same lemma
* freq_lex - an `int` feature of `lex` and `word` that gives a lexeme's frequency

The additions are made by processing the initial Tischendorf dataset alongside Text-Fabric's `modify` class.

### Load the Corpus

In [9]:
TF = Fabric(locations=output_dir)
api = TF.load('''

book chapter verse para
ketiv anlex_lem str_lem
strongs

''')

classes = api.makeAvailableIn(globals())

This is Text-Fabric 7.8.7
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

17 features found and 0 ignored
  0.00s loading features ...
   |     0.06s T otype                from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.30s T oslots               from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
   |     0.41s T qere                 from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.02s T verse                from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.00s T chapter              from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.00s T book                 from /Users/cody/github/tischendorf_tf/tf/2.8
   |      |     0.01s C __levels__           from otype, oslots, otext
   |      |     0.94s C __order__            from otype, oslots, __levels__
   |      |     0.06s C __rank__             from otype, __order__
   |      |     0.90s C __levUp__  

### Add glosses

In [10]:
glossary_source = '/Users/cody/github/eliranwong/OpenGNT/Glossary/'
glossary = os.path.join(glossary_source, 'GK_lemma_EnglishGloss.csv')
glossary_orig = '/Users/cody/github/billmounce/dictionary/dictionary.txt'
nodeFeatures = collections.defaultdict(lambda:collections.defaultdict(list))

Get latest openGNT data from Eliran Wong. He provides a list of abbreviated glosses at: https://github.com/eliranwong/OpenGNT/blob/master/Glossary/GK_lemma_EnglishGloss.csv 

The glosses themselves come from Bill Mounce:
https://github.com/billmounce/dictionary

The gloss matching is facilitated by Mounce's inclusion of strongs numbers.

In [11]:
!cd $glossary_source; git pull origin master

From https://github.com/eliranwong/OpenGNT
 * branch            master     -> FETCH_HEAD
Already up to date.


#### Load Glossary and Match Words to Glosses

Eliran Wong's glosses must be brought back to into contact with the original source for Strongs numbers.

In [12]:
with open(glossary, 'r') as infile:
    reader = csv.reader(infile)
    raw_glosses = [line[0].split('\t') for line in reader]
    glosses = {int(line[0]):line[2] for line in raw_glosses}
    surfaces2gloss = {line[1]:line[2] for line in raw_glosses}

with open('/Users/cody/github/billmounce/dictionary/dictionary.txt', 'r') as infile:
    mounce_dict = infile.read().split('\n')

The raw mounce dictionary.txt file is parsed since some strongs numbers appear to be missing from the .json version.

In [13]:
# process Mounce file 
# the file contains Mounce ID to strongs mappings

strongs2mounce = {}
find_ids = re.compile(r'^GK (G\d\d*) \| S ([G, π, 0-9]*)') # first line of dictionary entry
clean_ids = re.compile(r'\d\d*') # grabs the IDs

# parse Mounce dictionary
# add strongs data
for line in mounce_dict[100:]:
    
    # parse lines that contain data
    get_codes = find_ids.findall(line)
    if get_codes:
        
        mounce_id = int(clean_ids.findall(get_codes[0][0])[0]) # get mounce code
        strongs = [int(code.strip()) for code in clean_ids.findall(get_codes[0][1])] # get strongs numbers
        
        # make mapping from strongs to mounce
        for strong in strongs:
            strongs2mounce[strong] = mounce_id
            
# make the mapping from strong to mounce glosses
strongs2gloss = {strong:glosses[mounce] for strong, mounce in strongs2mounce.items()} # strongs code to Mounce gloss

Now make the matches

In [15]:
def match_gloss(word):
    '''
    Match a word node to its gloss.
    '''
    strongs = int(F.strongs.v(word))
    gloss = strongs2gloss.get(strongs, None) # first try strongs
    gloss = gloss or surfaces2gloss.get(F.anlex_lem.v(word), None) # second try the anlex surface form
    gloss = gloss or surfaces2gloss.get(F.str_lem.v(word), None) # third try the strongs surface form
    return gloss


no_matches = []

for word in F.otype.s('word'):
    
    gloss = match_gloss(word)
    
    if gloss:
        nodeFeatures['gloss'][word] = gloss
    else:
        nodeFeatures['gloss'][word] = 'UNKNOWN'
        no_matches.append((word, F.anlex_lem.v(word), F.strongs.v(word)))
        
print(f'{len(no_matches)} words not matched...')
print(len(nodeFeatures['gloss']), ' words matched...')

42 words not matched...
137711  words matched...


#### No Matches

A number of words receive no match on any of the criteria. Here they are:

In [17]:
no_matches

[(6544, 'σκορπίζω', '4650'),
 (14538, 'σκοτίζω', '4654'),
 (27124, 'σκοτίζω', '4654'),
 (30752, 'ἔγκυος', '1471'),
 (35506, 'Χουζᾶς', '5529'),
 (38904, 'σκορπίζω', '4650'),
 (41336, 'ἀνώτερος', '511'),
 (45261, 'ἐγκάθετος', '1455'),
 (50616, 'Σαλείμ', '4530'),
 (56788, 'σκορπίζω', '4650'),
 (59408, 'ἀπάρτι', '534'),
 (59848, 'ἀπάρτι', '534'),
 (61360, 'σκορπίζω', '4650'),
 (61905, 'χείμαρρος', '5493'),
 (64988, 'Ἁκελδαμάχ', '184'),
 (75735, 'Ἀθῆναι', '116'),
 (75755, 'Ἀθῆναι', '116'),
 (76126, 'Ἀθῆναι', '116'),
 (77637, 'Χίος', '5508'),
 (83368, 'σκοτίζω', '4654'),
 (83653, 'ἀμετανόητος', '279'),
 (87783, 'σκοτίζω', '4654'),
 (90306, 'Χλόη', '5514'),
 (91742, 'συναναμίγνυμι', '4874'),
 (91770, 'συναναμίγνυμι', '4874'),
 (96920, 'μαράνα θᾶ', '3134'),
 (96921, 'μαράνα θᾶ', '3134'),
 (99830, 'σκορπίζω', '4650'),
 (103687, 'ἄμωμος', '299'),
 (105544, 'ἄμωμος', '299'),
 (106738, 'ἄμωμος', '299'),
 (108039, 'ἄμωμος', '299'),
 (109845, 'Ἀθῆναι', '116'),
 (111477, 'συναναμίγνυμι', '4874'),
 (1

### Add Lexeme Objects

In [18]:
# Lexeme objects are generated by grouping words with identical
# lemma strings. The strings are mapped to a dict, lexstring2node,
# which guides the grouping process. Along the way, a mapping is 
# kept to lexeme's contained words (lex_slots) and their string feature

lexstring2node = {}
lex_slots = collections.defaultdict(set)
lex_features = collections.defaultdict(lambda: collections.defaultdict())
max_node = 0

# built the lexeme objects
for word in F.otype.s('word'):
    
    lemma = F.anlex_lem.v(word) # get Analytical lexeme string for mapping
    try:
        gloss = match_gloss(word)
    except:
        raise Exception(word)
    node = lexstring2node.get(lemma, None) # get assigned node number
    
    # assign node number if not exists
    if not node:
        max_node += 1
        node = max_node
        lexstring2node[lemma] = node # assign string 2 node mapping
        lex_features['anlex_lem'][node] = lemma # assign string feature to lexeme node
        lex_features['gloss'][node] = gloss # assign gloss
        
    # collect slot
    lex_slots[node].add(word)
    
len(lex_slots)

5394

Now the `freq_lex` feature is created. It is stored only on `lex` for now.

In [19]:
for node, words in lex_slots.items():
    lex_features['freq_lex'][node] = len(words)

We use the `modify` class from Text-Fabric (see [docs](https://annotation.github.io/text-fabric/Api/Compose/#modify)) to add new objects and features. The class requires a dictionary, called `addTypes`. That is formatted below before calling the modify class. Another dictionary, `metaData` gives the metadata for the new features.

In [20]:
addTypes = {'lex': {'nodeFrom': 1, # starting temporary node number
                    'nodeTo': max_node, # ending temp. node num.
                    'nodeSlots': lex_slots,
                    'nodeFeatures': lex_features,
                   }
           }

feature_meta = {
            'freq_lex': {'description': 'A lexeme\'s frequency throughout the corpus',
                         'valueType': 'int'},
            'gloss': {'Source': 'https://github.com/eliranwong/OpenGNT/blob/master/Glossary/GK_lemma_EnglishGloss.csv',
                      'Author': 'Bill Mounce',
                      'Editor': 'Eliran Wong',
                      'description': 'Lemma glosses from Bill Mounce'}
           }

Now the new object is added.

In [21]:
# first clean out any old data
!cd ../tf/2.8/_temp; for file in *.tf; do rm $file; done

In [22]:
modify(
    output_dirs.format(version='2.8'), # input files
    output_dirs.format(version='2.8/_temp'), # output location
    addTypes = addTypes,
    featureMeta = feature_meta,
    addFeatures = {'nodeFeatures':nodeFeatures}
)

  0.00s preparing and checking ...
This is Text-Fabric 7.8.7
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

17 features found and 0 ignored
  0.00s loading features ...
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
  0.25s All features loaded/computed - for details use loadLog()
  0.00s loading features ...
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
  0.00s All features loaded/computed - for details use loadLog()
  0.00s loading features ...
   |     0.00s T book_code            from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.02s T freq_lex             from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.30s T morph                from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.36s T vrsnum               from /Users/cody/github/tischendorf_tf/tf/2.8
  0.80s All additional features loaded - for details use loadLog()
   |     1.09s done
  1.09s add featu

True

### Move temporary files into the permanent position

In [23]:
!cd ../tf/2.8/_temp; for file in *.tf; do cp $file ../; done

## Reload and Test the Dataset

Do a test load of the new data.

In [24]:
TF = Fabric(locations=output_dir)
api = TF.load('''

book chapter verse para
ketiv anlex_lem str_lem
strongs gloss

''')

classes = api.makeAvailableIn(globals())

This is Text-Fabric 7.8.7
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

17 features found and 0 ignored
  0.00s loading features ...
   |     0.07s T otype                from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.52s T oslots               from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
   |     0.43s T qere                 from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.02s T verse                from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.00s T chapter              from /Users/cody/github/tischendorf_tf/tf/2.8
   |     0.00s T book                 from /Users/cody/github/tischendorf_tf/tf/2.8
   |      |     0.01s C __levels__           from otype, oslots, otext
   |      |     1.06s C __order__            from otype, oslots, __levels__
   |      |     0.09s C __rank__             from otype, __order__
   |      |     1.35s C __levUp__  

### Test Gloss

In [25]:
print('{:>20} {:>20} {:>20}\t{}'.format('node', 'text', 'lemma', 'gloss'))
for verse in list(F.otype.s('verse'))[:5]:
    for word in L.d(verse, 'word'):
        text = T.text(word)
        lemma = F.anlex_lem.v(word)
        gloss = F.gloss.v(word)
        print('{:>20} {:>20} {:>20}\t{}'.format(word, text, lemma, gloss))

                node                 text                lemma	gloss
                   1              Βίβλος                βίβλος	book
                   2            γενέσεως               γένεσις	birth
                   3               Ἰησοῦ                Ἰησοῦς	Jesus
                   4             Χριστοῦ               Χριστός	Christ
                   5                υἱοῦ                  υἱός	son
                   6              Δαυεὶδ                 Δαυίδ	David
                   7                υἱοῦ                  υἱός	son
                   8             Ἀβραάμ.                Ἀβραάμ	Abraham
                   9              Ἀβραὰμ                Ἀβραάμ	Abraham
                  10           ἐγέννησεν                γεννάω	to become the father of
                  11                 τὸν                     ὁ	the
                  12              Ἰσαάκ,                 Ἰσαάκ	Isaac
                  13               Ἰσαὰκ                 Ἰσαάκ	Isaac
                  

In [26]:
show_book = T.nodeFromSection(('1_John',))

for verse in L.d(show_book, 'verse'):
    
    print('{} {}:{}'.format(*T.sectionFromNode(verse)))
    print('\t\t', T.text(verse))

1_John 1:1
		 Ὃ ἦν ἀπ’ ἀρχῆς, ὃ ἀκηκόαμεν, ὃ ἑωράκαμεν τοῖς ὀφθαλμοῖς ἡμῶν, ὃ ἐθεασάμεθα καὶ αἱ χεῖρες ἡμῶν ἐψηλάφησαν, περὶ τοῦ λόγου τῆς ζωῆς 
1_John 1:2
		 καὶ ἡ ζωὴ ἐφανερώθη, καὶ ἑωράκαμεν καὶ μαρτυροῦμεν καὶ ἀπαγγέλλομεν ὑμῖν τὴν ζωὴν τὴν αἰώνιον ἥτις ἦν πρὸς τὸν πατέρα καὶ ἐφανερώθη ἡμῖν 
1_John 1:3
		 ὃ ἑωράκαμεν καὶ ἀκηκόαμεν ἀπαγγέλλομεν καὶ ὑμῖν, ἵνα καὶ ὑμεῖς κοινωνίαν ἔχητε μεθ’ ἡμῶν. καὶ ἡ κοινωνία δὲ ἡ ἡμετέρα μετὰ τοῦ πατρὸς καὶ μετὰ τοῦ υἱοῦ αὐτοῦ Ἰησοῦ Χριστοῦ. 
1_John 1:4
		 καὶ ταῦτα γράφομεν ἡμεῖς ἵνα ἡ χαρὰ ἡμῶν ᾖ πεπληρωμένη. 
1_John 1:5
		 Καὶ ἔστιν αὕτη ἡ ἀγγελία ἣν ἀκηκόαμεν ἀπ’ αὐτοῦ καὶ ἀναγγέλλομεν ὑμῖν, ὅτι ὁ θεὸς φῶς ἐστιν καὶ σκοτία ἐν αὐτῷ οὐκ ἔστιν οὐδεμία. 
1_John 1:6
		 ἐὰν εἴπωμεν ὅτι κοινωνίαν ἔχομεν μετ’ αὐτοῦ καὶ ἐν τῷ σκότει περιπατῶμεν, ψευδόμεθα καὶ οὐ ποιοῦμεν τὴν ἀλήθειαν· 
1_John 1:7
		 ἐὰν δὲ ἐν τῷ φωτὶ περιπατῶμεν ὡς αὐτός ἐστιν ἐν τῷ φωτί, κοινωνίαν ἔχομεν μετ’ ἀλλήλων καὶ τὸ αἷμα Ἰησοῦ τοῦ υἱοῦ αὐτοῦ καθαρίζει ἡμᾶς ἀπὸ πάσης ἁμαρτίας. 


In [27]:
T.sectionFromNode(100535)

('2_Corinthians', 11, 17)

In [28]:
for para in L.d(show_book, 'paragraph'):
    print(F.book.v(show_book), '§', F.para.v(para))
    
    for verse in L.d(para, 'verse'):
        book,chapter,vrs = T.sectionFromNode(verse)
        print(f'({chapter}:{vrs}) {T.text(verse)}', end='')
    print('\n')

1_John § 1
(1:1) Ὃ ἦν ἀπ’ ἀρχῆς, ὃ ἀκηκόαμεν, ὃ ἑωράκαμεν τοῖς ὀφθαλμοῖς ἡμῶν, ὃ ἐθεασάμεθα καὶ αἱ χεῖρες ἡμῶν ἐψηλάφησαν, περὶ τοῦ λόγου τῆς ζωῆς (1:2) καὶ ἡ ζωὴ ἐφανερώθη, καὶ ἑωράκαμεν καὶ μαρτυροῦμεν καὶ ἀπαγγέλλομεν ὑμῖν τὴν ζωὴν τὴν αἰώνιον ἥτις ἦν πρὸς τὸν πατέρα καὶ ἐφανερώθη ἡμῖν (1:3) ὃ ἑωράκαμεν καὶ ἀκηκόαμεν ἀπαγγέλλομεν καὶ ὑμῖν, ἵνα καὶ ὑμεῖς κοινωνίαν ἔχητε μεθ’ ἡμῶν. καὶ ἡ κοινωνία δὲ ἡ ἡμετέρα μετὰ τοῦ πατρὸς καὶ μετὰ τοῦ υἱοῦ αὐτοῦ Ἰησοῦ Χριστοῦ. (1:4) καὶ ταῦτα γράφομεν ἡμεῖς ἵνα ἡ χαρὰ ἡμῶν ᾖ πεπληρωμένη. 

1_John § 2
(1:5) Καὶ ἔστιν αὕτη ἡ ἀγγελία ἣν ἀκηκόαμεν ἀπ’ αὐτοῦ καὶ ἀναγγέλλομεν ὑμῖν, ὅτι ὁ θεὸς φῶς ἐστιν καὶ σκοτία ἐν αὐτῷ οὐκ ἔστιν οὐδεμία. (1:6) ἐὰν εἴπωμεν ὅτι κοινωνίαν ἔχομεν μετ’ αὐτοῦ καὶ ἐν τῷ σκότει περιπατῶμεν, ψευδόμεθα καὶ οὐ ποιοῦμεν τὴν ἀλήθειαν· (1:7) ἐὰν δὲ ἐν τῷ φωτὶ περιπατῶμεν ὡς αὐτός ἐστιν ἐν τῷ φωτί, κοινωνίαν ἔχομεν μετ’ ἀλλήλων καὶ τὸ αἷμα Ἰησοῦ τοῦ υἱοῦ αὐτοῦ καθαρίζει ἡμᾶς ἀπὸ πάσης ἁμαρτίας. (1:8) ἐὰν εἴπωμεν ὅτι ἁμαρτίαν οὐκ ἔχομ